In [1]:
import pandas as pd

df = pd.read_csv('research_traffic_multiple_labels.csv')
df.shape

(60000, 12)

In [2]:
print(df.dtypes)
print(df['Label'].value_counts()[1])
print(df['Label'].value_counts()[0])
df.head()

Count of Source IP        int64
Port Count                int64
Pair Count Ratio        float64
Packet Count Diff         int64
Lookup Count Diff         int64
Protocol                  int64
Average Packet Count    float64
Average Byte Count      float64
Packet Std Dev          float64
Byte Std Dev            float64
Duration per Flow       float64
Label                     int64
dtype: object
10000
10000


,Count of Source IP,Port Count,Pair Count Ratio,Packet Count Diff,Lookup Count Diff,Protocol,Average Packet Count,Average Byte Count,Packet Std Dev,Byte Std Dev,Duration per Flow,Label
0,545,545,0.0,2,43,6,0.027523,4.788991,0.163601,28.466647,0.668020,3
1,545,545,0.0,10,0,6,0.009174,1.596330,0.095342,16.589551,0.419081,3
2,52,52,0.0,3,0,6,0.000000,0.000000,0.000000,0.000000,0.018792,3
3,545,545,0.0,13,21,6,0.023853,4.150459,0.355016,61.772825,2.946281,3
4,545,545,0.0,2,131,6,0.027523,4.788991,0.163601,28.466647,2.674461,3


In [3]:
# obj_cols=df.dtypes[df.dtypes == "object"].index.values.tolist()
# print(obj_cols)

# from sklearn.preprocessing import LabelEncoder
# #Encode labels of multiple columns at once

# df[obj_cols] = df[obj_cols].astype(str)
# df[obj_cols] = df[obj_cols].apply(LabelEncoder().fit_transform)
# #
# # Print head
# #
# print(df.dtypes)
# df.head()

In [4]:
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.utils import to_categorical
from datetime import datetime

y = df.Label.values  # Ensure y is a NumPy array before encoding
X = df.drop(['Label'], axis=1)

# Normalize the dataset using MinMaxScaler
normalizer = MinMaxScaler()
X_normalized = normalizer.fit_transform(X)
X = pd.DataFrame(X_normalized, columns=X.columns)
print("Dataset normalized using MinMaxScaler.")

# Ensure y is one-hot encoded for multiclass classification
y = to_categorical(y)

X_train_cv, X_unseen_test, y_train_cv, y_unseen_test = train_test_split(X, y, test_size=0.2, random_state=42)

kf = KFold(n_splits=10, shuffle=True, random_state=42)

def create_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train_cv.shape[1], activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(6, activation='softmax'))  # Adjusted for multiclass classification
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
    return model


Dataset normalized using MinMaxScaler.


In [5]:
# def create_model():
#     model = Sequential()
#     model.add(Dense(32, input_dim=X_train_cv.shape[1], activation='relu'))
#     model.add(Dense(16, activation='relu'))
#     model.add(Dense(1, activation='sigmoid'))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model

In [6]:
# Initialize lists to store metrics
train_accuracies = []
test_accuracies = []
precisions = []
recalls = []
f1_scores = []
confusion_matrices = []
start = datetime.now()

for train_index, test_index in kf.split(X_train_cv):
    X_train, X_test = X_train_cv.iloc[train_index], X_train_cv.iloc[test_index]
    y_train, y_test = y_train_cv[train_index], y_train_cv[test_index]  # Use NumPy indexing here
    
    model = create_model()
    model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0)
    
    # Evaluate on training data
    y_pred_train = (model.predict(X_train) > 0.5).astype(int)
    y_pred_test = (model.predict(X_test) > 0.5).astype(int)
    
    # Calculate metrics for the test fold
    train_accuracies.append(accuracy_score(y_train.argmax(axis=1), y_pred_train.argmax(axis=1)))
    test_accuracies.append(accuracy_score(y_test.argmax(axis=1), y_pred_test.argmax(axis=1)))
    precisions.append(precision_score(y_test.argmax(axis=1), y_pred_test.argmax(axis=1), average='weighted'))
    recalls.append(recall_score(y_test.argmax(axis=1), y_pred_test.argmax(axis=1), average='weighted'))
    f1_scores.append(f1_score(y_test.argmax(axis=1), y_pred_test.argmax(axis=1), average='weighted'))
    confusion_matrices.append(confusion_matrix(y_test.argmax(axis=1), y_pred_test.argmax(axis=1)))

y_unseen_pred = (model.predict(X_unseen_test) > 0.5).astype(int)
unseen_accuracy = accuracy_score(y_unseen_test.argmax(axis=1), y_unseen_pred.argmax(axis=1))
unseen_precision = precision_score(y_unseen_test.argmax(axis=1), y_unseen_pred.argmax(axis=1), average='weighted')
unseen_recall = recall_score(y_unseen_test.argmax(axis=1), y_unseen_pred.argmax(axis=1), average='weighted')
unseen_f1 = f1_score(y_unseen_test.argmax(axis=1), y_unseen_pred.argmax(axis=1), average='weighted')
unseen_conf_matrix = confusion_matrix(y_unseen_test.argmax(axis=1), y_unseen_pred.argmax(axis=1))
end = datetime.now()

C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1350/1350 ━━━━━━━━━━━━━━━━━━━━ 1s 533us/step
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 440us/step


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1350/1350 ━━━━━━━━━━━━━━━━━━━━ 1s 637us/step
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

1350/1350 ━━━━━━━━━━━━━━━━━━━━ 1s 847us/step
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1350/1350 ━━━━━━━━━━━━━━━━━━━━ 1s 928us/step
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1350/1350 ━━━━━━━━━━━━━━━━━━━━ 1s 857us/step
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 609us/step


In [7]:
# Calculate the average confusion matrix for multi-class classification
import numpy as np

# Determine the number of classes from the confusion matrices
num_classes = confusion_matrices[0].shape[0]

# Initialize an array to store the sum of confusion matrices
confusion_matrix_sum = np.zeros((num_classes, num_classes))

# Sum up the confusion matrices from all folds
for cm in confusion_matrices:
    confusion_matrix_sum += cm

# Compute the average confusion matrix
average_conf_matrix = confusion_matrix_sum / len(confusion_matrices)

# Display the average confusion matrix
print("Average Confusion Matrix:")
max_width = max(len("{:.2f}".format(value)) for row in average_conf_matrix for value in row)

# Print each row with formatted values
for row in average_conf_matrix:
    print(" ".join(f"{value:>{max_width}.2f}" for value in row))

Average Confusion Matrix:
547.50  35.50   0.00  40.40   0.40 175.30
 16.30 749.90   0.00   0.00   0.00  28.20
  5.70   0.00 793.80   0.00   0.00   0.00
  0.10   0.00   0.00 798.90   0.00   0.60
  4.00   0.00   0.00   0.00 799.50   0.00
  4.70   0.00   0.00   0.00   0.00 799.20


In [8]:
# Print the metrics for each fold
for i in range(10):
    print(f"Fold {i+1}:")
    print(f"Training Accuracy: {train_accuracies[i]}")
    print(f"Testing Accuracy: {test_accuracies[i]}")
    print(f"Precision: {precisions[i]}")
    print(f"Recall: {recalls[i]}")
    print(f"F1-Score: {f1_scores[i]}")
    print(f"Confusion Matrix:\n{confusion_matrices[i]}\n")

print(f"Average Training Accuracy: {sum(train_accuracies) / 10}")
print(f"Average Testing Accuracy: {sum(test_accuracies) / 10}")
print(f"Average Precision: {sum(precisions) / 10}")
print(f"Average recall: {sum(recalls) / 10}")
print(f"Average F1 Score: {sum(f1_scores) / 10}")

# Print the performance on the unseen data
print("Performance on Unseen Data:")
print(f"Accuracy: {unseen_accuracy}")
print(f"Precision: {unseen_precision}")
print(f"Recall: {unseen_recall}")
print(f"F1-Score: {unseen_f1}")
print(f"Confusion Matrix:\n{unseen_conf_matrix}")
print("excution time: ", end-start)
 

Fold 1:
Training Accuracy: 0.9352083333333333
Testing Accuracy: 0.935625
Precision: 0.941308756107088
Recall: 0.935625
F1-Score: 0.9331087281212704
Confusion Matrix:
[[513  42   0  33   0 173]
 [ 16 774   0   0   0  30]
 [  6   0 767   0   0   0]
 [  0   0   0 794   0   0]
 [  4   0   0   0 803   0]
 [  5   0   0   0   0 840]]

Fold 2:
Training Accuracy: 0.9361111111111111
Testing Accuracy: 0.93
Precision: 0.9375208281161953
Recall: 0.93
F1-Score: 0.927695692975077
Confusion Matrix:
[[560  31   0  47   0 191]
 [ 22 725   0   0   0  30]
 [  6   0 806   0   0   0]
 [  0   0   0 813   0   1]
 [  5   0   0   0 780   0]
 [  3   0   0   0   0 780]]

Fold 3:
Training Accuracy: 0.9355787037037037
Testing Accuracy: 0.9345833333333333
Precision: 0.9418167400243974
Recall: 0.9345833333333333
F1-Score: 0.9324664778959777
Confusion Matrix:
[[538  33   0  37   0 182]
 [ 14 802   0   0   0  31]
 [  9   0 765   0   0   0]
 [  0   0   0 823   0   2]
 [  2   0   0   0 767   0]
 [  4   0   0   0   0 791]

In [9]:
# import pickle
# with open('ann_multi.pkl', 'wb') as file:
#     pickle.dump(model, file)